# Part 4: Control Flow
# 14. 반복할 수 있는 객체, 반복자, 제너레이터

## Index
- 14.1 Sentence 버전 #1: 단어 시퀀스
- 14.2 반복형과 반복자
- 14.3 Sentence 버전 #2: 고전적인 반복자
- 14.4 Sentence 버전 #3: 제너레이터 함수
- 14.5 Sentence 버전 #4: 느긋한 구현
- 14.6 Sentence 버전 #5: 제너레이터 표현식
- 14.7 제너레이터 표현식 : 언제 사용하나?
- 14.8 또 다른 예제 : 등차수열 제너레이터
- 14.9 표준 라이브러리의 제너레이터 함수
- 14.10 파이썬 3.3의 새로운 구문 : yield from
- 14.11 반복형을 리듀스하는 함수
- 14.12 iter() 함수 들여다보기
- 14.13 사례 연구 : 데이터베이스 변환 유틸리티 안의 제너레이터
- 14.14 코루틴으로서의 제너레이터
- 14.15 요약

## 14.1 Sentence 버전 #1: 단어 시퀀스
`Sentence` 객체는 입력 받은 텍스트를 단어 단위로 반복하는 객체. 시작은 `Sequence`를 이용하여! `Sequence`는 iterable하니까!

In [1]:
#Example 14-1
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

In [2]:
#Example 14-2
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [3]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [4]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

왜 `Sequence`는 iterable일까? 인터프리터가 객체 x를 반복하면 `iter(x)`를 자동으로 호출한다. `iter` 내장 함수는:  
1. 객체가 `__iter__`를 구현했는지 확인하고 반복자를 얻기 위해 호출한다.
2. `__iter__`가 구현 되어있지 않고, `__getitem__`이 구현되어 있으면, 인덱스 0부터 순차적으로 아이템을 가져오는 방식으로 반복자를 만들어낸다.
3. 만약 실패하면, 파이썬은 `TypeError`를 발생하고 대체로 "C object is not iterable,"이라고 말한다.

이게 `Sequence`가 iterable인 이유. `__getitem__`을 가지고 있고 표준 시퀀스는 `__iter__`까지 구현되어 있음. 이것은 duck typing의 극한의 형태. 이걸 goose typing의 관점에서 보면 심플하지만 유연하지 않음. 단순히 `__iter__`이 구현되어 있으면 `iterable`이라고 하면 되기 때문. 서브클래싱이나 등록은 필요하지 않음. `abc.Iterable`에 `__subclasshook__`이 이미 구현되어 있기 때문.

In [5]:
class Foo:
    def __iter__(self):
        pass
    
from collections import abc
issubclass(Foo, abc.Iterable)

True

In [6]:
f = Foo()
isinstance(f, abc.Iterable)

True

보면 정상적으로 서브클래스, 인스턴스로 인식하는 것을 볼 수 있다. 하지만 #14-1은?

In [7]:
issubclass(Sentence, abc.Iterable)

False

### 응 안돼

객체가 iterable인지 명시적으로 확인하고 바로 객체를 iterate하면 의미가 없다. 왜냐하면 어짜피 `TypeError`가 발생하기 때문! 명시적으로 체크하기보다 `TypeError`를 `try/except`로 처리하는게 나을 수도 있다. 나중에 iteration을 하고 싶은 거면 미리 오류를 잡는 거기 때문에 명시적으로 확인하는게 의미가 있다.

## 14.2 반복형과 반복자
파이썬은 iterable에서 iterator를 얻는다.

In [8]:
s = 'ABC'
it = iter(s)
while True:
    try:
        print(next(it)) #next available item
    except StopIteration: # signal that the iterator is exhausted
        del it
        break

A
B
C


![fig14-1](../images/fig_14-1.png)

In [9]:
#Example 14-3
from abc import *

class Iterator(abc.Iterable):
    
    __slots__ = ()
    
    @abstractmethod
    def __next__(self):
        'Return the next item from the iterator. When exhausted, raise StopIteration'
        raise StopIteration
        
    def __iter__(self):
        return self
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterator:
            if (any("__next__" in B.__dict__ for B in C.__mro__) and
                any("__iter__" in B.__dict__ for B in C.__mro__)):
                return true
        return NotImplemented

In [10]:
s3 = Sentence('Pig and Pepper')
it = iter(s3)
next(it)

'Pig'

In [11]:
next(it)

'and'

In [12]:
next(it)

'Pepper'

In [13]:
next(it)

StopIteration: 

`iterator`는 `__next__`와 `__iter__`만 요구하기 때문에 `next()`를 돌려보면서 `StopIteration`이 나올 때까지 보는 것밖에 아이템이 남았는지 확인할 수 있는 방법이 없다. 그리고 빠꾸도 없기 때문에 다시 시작하려면 `iter()`에 `iterable`을 넣어서 `iterator`를 만들어야 함.

## 14.3 Sentence 버전 #2: 고전적인 반복자

In [14]:
#Example 14-4
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        return SentenIterator(self.words)
    

class SentenceIterator:

    def __init__(self, words):
        self.words = words
        self.index = 0
        
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word
    
    def __iter__(self):
        return self

In [15]:
issubclass(SentenceIterator, abc.Iterator)

True

정리를 해보자!  
`iterable` : 매번 새로운 `iterator`를 객체화하는 `__iter__` 함수를 가짐<br>
`iterator` : 각각의 아이템을 반환하기 위한 `__next__`, 자기 자신을 반환하는 `__iter__` 메소드를 가짐.  

<blockquote>iterator는 iterable이지만 iterable은 iterator가 아니다.</blockquote>

`Sentence`에 `__next__`를 구현해보면 어떨까? 각각의 `Sentence` 인스턴스가 동시에 `iterable`이고 자신을 순환하는 `iterator`가 될까? 안된다!  
다중 순회를 지원하기 위해서는 `iter(my_iterable)`이 호출될 때 새롭고 독립적인 `iterator`를 만들어내야 한다. 

## 14.4 Sentence 버전 #3: 제너레이터 함수
`SentenceIterator`를 제너레이터로 대체해보자. 

In [16]:
#Example 14-5
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for word in self.words:
            yield word
        return

In [17]:
issubclass(Sentence, abc.Iterable)

True

### Generator는 어떻게 작동할까?
일단 `yield`를 사용하면 generator이다. 호출되면 generator 객체를 반환한다. 

In [18]:
def gen_123():
    yield 1
    yield 2
    yield 3
gen_123

<function __main__.gen_123()>

In [19]:
gen_123()

<generator object gen_123 at 0x7f24c0b9ffc0>

In [20]:
for i in gen_123():
    print(i)

1
2
3


In [21]:
g = gen_123()
next(g)

1

In [22]:
next(g)

2

In [23]:
next(g)

3

In [24]:
next(g)

StopIteration: 

A generator function builds a generator object that wraps the body of the function. When we invoke next(…) on the generator object, execution advances to the next yield in the function body, and the next(…) call evaluates to the value yielded when the function body is suspended. Finally, when the function body returns, the enclosing generator object raises StopIteration, in accordance with the Iterator protocol.

In [25]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')
    
g = iter(gen_AB())

In [26]:
test = []
for i in g:
    test.append(i)

start
continue
end.


In [27]:
test

['A', 'B']

## 14.5 Sentence 버전 #4: 느긋한 구현
lazy의 좋은 점: 메모리를 아낄 수 있다. 고작 몇 개 iterate 하자고 전체 사이즈의 list를 만들어내고 시작하는 것은 낭비다. `re.findall` -> `re.finditer`로 바꾸면 메모리를 아낄 수 있다!

In [28]:
#Example 14-7
import re
import reprlib

RE_WORD = re.compile('\w+')


class Sentence:
    
    def __init__(self, next):
        self.text = text
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for match in RE_WORD.finditer(self.text):
            yield match.group()

## 14.6 Sentence 버전 #5: 제너레이터 표현식
제너레이터 표현식을 사용하면 더 짧게 표현할 수 있다. 제너레이터 표현식은 list comprehension의 lazy version이다. 제너레이터는 on demand로 lazy하게 아이템을 생성한다.

In [29]:
#Example 14-8
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')
    
res1 = [x*3 for x in gen_AB()]

start
continue
end.


In [30]:
for i in res1:
    print('-->', i)

--> AAA
--> BBB


In [31]:
res2 = (x*3 for x in gen_AB())
#소비된 결과가 아니라 generator를 return
res2

<generator object <genexpr> at 0x7f24c0bac2b0>

In [32]:
for i in res2: #for 
    print('-->', i)

start
--> AAA
continue
--> BBB
end.


In [33]:
#Example 14-9
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

달라진 것은 `__iter__()` 뿐이다! `yield`는 없어졌지만 결과는 동일!

## 14.7 제너레이터 표현식 : 언제 사용하나?
일단은 앞에서 봤던 예들을 들 수가 있다. `Vector` class 선언할 때 사용했었다. `__eq__`, `__hash__`, `__abs__`,.. 이런 것들. 함수를 따로 선언하지 않고, 구문적으로 짧게 쓸 때도 사용함.(#14.9) 하지만 제너레이터 함수가 더 유연함. 당연한 얘기! 표현식 안에는 많이 넣으면 일단 해석이 어려워지고 지저분해진다. shortcut을 사용하려고 사용한 표현식이 shortcut이 아닐 수도 있다..! 표현식은 2X 레버리지 같은 존재. 심플한 건 더 심플하게, 복잡한 건 더 복잡하게 만들어버린다고 생각하면 좋을듯! 필자는 한 줄을 넘어가면 함수로 구현한다고 한다.

## 14.8 또 다른 예제 : 등차수열 제너레이터

In [34]:
#Example 14-11
class ArithmeticProgression:
    
    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step
        self.end = end
        
    def __iter__(self):
        result = type(self.begin + self.step)(self.begin)
        #step의 type으로 강제된 self.begin의 값을 result에 저장
        forever = self.end is None
        index = 0
        while forever or result < self.end:
            yield result
            index += 1
            result = self.begin + self.step * index
            #소수점을 누적하면서 생기는 문제를 막기 위해 매번 새로 계산해줌.

In [35]:
#Example 14-10
ap = ArithmeticProgression(0, 1, 3)
list(ap)

[0, 1, 2]

In [36]:
ap

In [37]:
ap = ArithmeticProgression(1, .5, 3)
list(ap)

[1.0, 1.5, 2.0, 2.5]

In [38]:
ap = ArithmeticProgression(0, 1/3, 1)
list(ap)

[0.0, 0.3333333333333333, 0.6666666666666666]

In [39]:
from fractions import Fraction
ap = ArithmeticProgression(0, Fraction(1, 3), 1)
list(ap)

[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]

In [40]:
from decimal import Decimal
ap = ArithmeticProgression(0, Decimal('.1'), .3)
list(ap)

[Decimal('0'), Decimal('0.1'), Decimal('0.2')]

In [41]:
#Example 14-12
def aritprog_gen(begin, step, end=None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or result < end:
        yield result
        index += 1
        result = begin + step * index

자, 이제 잘 만들어 놓은 거(itertools)를 갖다 써보자.
### Arithmetic Progression with itertools

In [42]:
import itertools
gen = itertools.count(1, .5)

In [43]:
next(gen), next(gen), next(gen)

(1, 1.5, 2.0)

하지만 이건 끝이 없기 때문에
```Python
for i in gen:
    print(i)
```
같은 걸 한다던가,
`list(gen)` 같은 것을 해버리면, 가용 메모리보다 큰 작업을 하기 때문에 실패한다. for문은 계속 돌겠지만..?  
그러면 이제 `itertools.takewhile`을 써보자. 다른 generator를 소비해서 조건이 `False`가 될 때까지 돌려서 generator를 생성한다.

In [44]:
gen = itertools.takewhile(lambda n: n<3, itertools.count(1, .5))
list(gen)

[1, 1.5, 2.0, 2.5]

이걸 사용해서 Example 14-12에 적용해보자.

In [45]:
#Example 14-13
import itertools

def aritprog_gen(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is not None:
        ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
    return ap_gen

또, `yield`문은 사라졌지만 여전히 generator를 return한다. 결론은, 원리를 알고 있는 걸 잘 갖다 쓰자는 말!

## 14.9 표준 라이브러리의 제너레이터 함수
line-by-line iteration이나, `os.walk` 같은 멋있는 함수들이 있지만 좀 더 본질적인 얘기도 들어가보자. 하이레벨의 기능면에서 함수들을 나눌 수 있다.

### 1. Filtering Generator
입력 iterable로부터 부분의 값을 변경 없이 생성한다. 위에서 봤던 `itertools.takewhile` 같은 것이 포함된다. `takewhile`처럼 여기에 속하는 대부분의 함수들이 `predicate`를 가진다. 이것은 `input`이 `output`에 포함되는지를 판단하는 일변수 boolean 함수이다.
![table14-1](../images/table_14-1.PNG)

In [46]:
#Example 14-14
def vowel(c):
    return c.lower() in 'aeiou'

In [47]:
list(filter(vowel, 'Aardvark'))
#predicate에 맞는 것들을 return

['A', 'a', 'a']

In [48]:
import itertools
list(itertools.filterfalse(vowel, 'Aardvark'))
#predicate가 False인 것들을 yield

['r', 'd', 'v', 'r', 'k']

In [49]:
list(itertools.dropwhile(vowel, 'Aardvark'))

['r', 'd', 'v', 'a', 'r', 'k']

In [50]:
list(itertools.dropwhile(vowel, 'zAardvark'))
#predicate가 True일 때까지는 skip, 나머지는 그대로 yield

['z', 'A', 'a', 'r', 'd', 'v', 'a', 'r', 'k']

In [51]:
list(itertools.takewhile(vowel, 'Aardvark'))
#predicate가 True일 때가지만 돈다.

['A', 'a']

In [52]:
list(itertools.compress('Aardvark', (1,0,1,1,0,1)))
#selector_it이 true인 것만 yield

['A', 'r', 'd', 'a']

In [53]:
list(itertools.islice('Aardvark', 4))
#stop까지 slice해서 yield

['A', 'a', 'r', 'd']

In [54]:
list(itertools.islice('Aardvark', 4, 7))
#start부터 stop까지 slice해서 yield

['v', 'a', 'r']

In [55]:
list(itertools.islice('Aardvark', 1, 7, 2))
#start, stop, step

['a', 'd', 'a']

### 2. Mapping Generator
각각의 개별적인 input iterable로부터 계산된 값을 yield. inputdl 하나 보다 많은 iterable일 경우, output은 첫번째 iterable이 소진되면 멈춘다.
![table14-2](../images/table_14-2.PNG)

In [56]:
#Example 14-15
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
import itertools
list(itertools.accumulate(sample))
#default sum

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [57]:
list(itertools.accumulate(sample, min))
#running min

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [58]:
list(itertools.accumulate(sample, max))
#running max

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

In [59]:
import operator
list(itertools.accumulate(sample, operator.mul))
#running product

[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]

In [60]:
list(itertools.accumulate(range(1, 11), operator.mul))

[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

In [61]:
#Example 14-16
list(enumerate('albatroz', 1))

[(1, 'a'),
 (2, 'l'),
 (3, 'b'),
 (4, 'a'),
 (5, 't'),
 (6, 'r'),
 (7, 'o'),
 (8, 'z')]

In [62]:
import operator
list(map(operator.mul, range(11), range(11)))
#square

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [63]:
list(map(operator.mul, range(11), [2, 4, 8]))
#stop when shortest iterable ends

[0, 4, 16]

In [64]:
list(map(lambda a, b: (a, b), range(11), [2, 4, 8]))
#built-in zip does

[(0, 2), (1, 4), (2, 8)]

In [65]:
import itertools
list(itertools.starmap(operator.mul, enumerate('albatroz', 1)))
#func를 it의 각각에 적용

['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']

In [66]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
list(itertools.starmap(lambda a, b: b/a,
                       enumerate(itertools.accumulate(sample), 1)))
#average

[5.0,
 4.5,
 3.6666666666666665,
 4.75,
 5.2,
 5.333333333333333,
 5.0,
 4.375,
 4.888888888888889,
 4.5]

### Merging Generator
여러 개의 iterable들에서 값을 yield. `chain`과 `chain.from_iterable`은 순차적으로 iterable을 소비하고, `product`, `zip`, `zip_longest`는 병렬로 iterable을 소비한다.
![table14-3](../images/table_14-3.PNG)

In [67]:
#Example 14-17
list(itertools.chain('ABC', range(2)))
#chain은 두 개 이상의 iterable을 호출

['A', 'B', 'C', 0, 1]

In [68]:
list(itertools.chain(enumerate('ABC')))
#single input이면 아무것도 안함.

[(0, 'A'), (1, 'B'), (2, 'C')]

In [69]:
list(enumerate('ABC'))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [70]:
list(itertools.chain.from_iterable(enumerate('ABC')))
#하지만 from_iterable은 파괴왕. 다 찢어버린다.

[0, 'A', 1, 'B', 2, 'C']

In [71]:
list(zip('ABC', range(5)))
#앞에서 봤던 zip과의 재회. 김밥 말아서 썰어주는 함수

[('A', 0), ('B', 1), ('C', 2)]

In [72]:
list(zip('ABC', range(5), [10, 20, 30, 40]))
#제일 짧은 iterable이 끝나면 끝남

[('A', 0, 10), ('B', 1, 20), ('C', 2, 30)]

In [73]:
list(itertools.zip_longest('ABC', range(5)))
#zip_longest는 긴 iterable 기준. default는 None

[('A', 0), ('B', 1), ('C', 2), (None, 3), (None, 4)]

In [74]:
list(itertools.zip_longest('ABC', range(5), fillvalue='?'))
#fillvalue를 설정해주면 None 대신 넣어줌.

[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]

In [75]:
#Example 14-18
list(itertools.product('ABC', range(2)))
#Cartesian product

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]

In [76]:
suits = 'spades hearts diamonds clubs'.split()
list(itertools.product('AK', suits))

[('A', 'spades'),
 ('A', 'hearts'),
 ('A', 'diamonds'),
 ('A', 'clubs'),
 ('K', 'spades'),
 ('K', 'hearts'),
 ('K', 'diamonds'),
 ('K', 'clubs')]

In [77]:
list(itertools.product('ABC'))
#product에게 하나만 준 경우. 쓸모가 없다.

[('A',), ('B',), ('C',)]

In [78]:
list(itertools.product('ABC', repeat=2))
#그림자 분신술

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

In [79]:
list(itertools.product(range(2), repeat=3))

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [80]:
rows = itertools.product('AB', range(2), repeat=2)
for row in rows: print(row)

('A', 0, 'A', 0)
('A', 0, 'A', 1)
('A', 0, 'B', 0)
('A', 0, 'B', 1)
('A', 1, 'A', 0)
('A', 1, 'A', 1)
('A', 1, 'B', 0)
('A', 1, 'B', 1)
('B', 0, 'A', 0)
('B', 0, 'A', 1)
('B', 0, 'B', 0)
('B', 0, 'B', 1)
('B', 1, 'A', 0)
('B', 1, 'A', 1)
('B', 1, 'B', 0)
('B', 1, 'B', 1)


일부 제너레이터 함수들은 입력으로부터 값을 확장한다.
![table14-4](../images/table_14-4.PNG)

In [81]:
#Example 14-19
ct = itertools.count()
next(ct)

0

In [82]:
next(ct), next(ct), next(ct)
#default step=1

(1, 2, 3)

In [83]:
list(itertools.islice(itertools.count(1, .3), 3))
#step = 0.3

[1, 1.3, 1.6]

In [84]:
cy = itertools.cycle('ABC')
next(cy)

'A'

In [85]:
list(itertools.islice(cy, 7))

['B', 'C', 'A', 'B', 'C', 'A', 'B']

In [86]:
rp = itertools.repeat(7)
next(rp), next(rp)

(7, 7)

In [87]:
list(itertools.repeat(8, 4))

[8, 8, 8, 8]

In [88]:
list(map(operator.mul, range(11), itertools.repeat(5)))

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

In [89]:
#Example 14-20
list(itertools.combinations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [90]:
list(itertools.combinations_with_replacement('ABC', 2))
#중복 사용 허용

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

In [91]:
list(itertools.permutations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

In [92]:
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

### Rearranging Generator
input을 모조리 yield 하지만, 재배치하는 제너레이터에 대해 알아본다. `reversed`는 이 섹션에서 유일하게 모든 iterable이 아닌 `sequence`만을 입력으로 가진다. 왜냐하면 앞뒤의 순서를 바꾸는 건데 길이가 정해져 있어야 하는 것이 당연하기 때문!  
`itertools.groupby`는 input iterable이 그루핑 기준에 따라 정렬되어 있거나, 적어도 아이템들이 기준에 따라 군집화되어 있다고 가정한다 - 정렬되어 있지 않더라도.
![table14-5](../images/table_14-5.PNG)

In [93]:
#Example 14-21
list(itertools.groupby('LLLLAAGGG'))

[('L', <itertools._grouper at 0x7f24c0b57780>),
 ('A', <itertools._grouper at 0x7f24c0b57320>),
 ('G', <itertools._grouper at 0x7f24c0b57cc0>)]

In [94]:
for char, group in itertools.groupby('LLLLAAAGG'):
    print(char, '->', list(group))

L -> ['L', 'L', 'L', 'L']
A -> ['A', 'A', 'A']
G -> ['G', 'G']


In [95]:
animals = ['duck', 'eagle', 'rat', 'giraffe', 'bear', 'bat', 
           'dolphin', 'shark', 'lion']
animals.sort(key=len)
animals

['rat', 'bat', 'duck', 'bear', 'lion', 'eagle', 'shark', 'giraffe', 'dolphin']

In [96]:
for length, group in itertools.groupby(animals, len):
    print(length, '->', list(group))

3 -> ['rat', 'bat']
4 -> ['duck', 'bear', 'lion']
5 -> ['eagle', 'shark']
7 -> ['giraffe', 'dolphin']


In [97]:
for length, group in itertools.groupby(reversed(animals), len):
    print(length, '->', list(group))

7 -> ['dolphin', 'giraffe']
5 -> ['shark', 'eagle']
4 -> ['lion', 'bear', 'duck']
3 -> ['bat', 'rat']


`itertools.tee`는 하나의 입력 iterable에서 여러 개의 제너레이터로 값을 yield

In [98]:
list(itertools.tee('ABC'))

[<itertools._tee at 0x7f24c0b55108>, <itertools._tee at 0x7f24c0b55588>]

In [99]:
g1, g2 = itertools.tee('ABC')

In [100]:
next(g1)

'A'

In [101]:
next(g2)

'A'

In [102]:
next(g2)

'B'

In [103]:
list(g1)

['B', 'C']

In [104]:
list(g2)

['C']

In [105]:
list(zip(*itertools.tee('ABC')))

[('A', 'A'), ('B', 'B'), ('C', 'C')]

## 14.10 파이썬 3.3의 새로운 구문 : yield from
제너레이터가 다른 제너레이터로부터 값을 받아서 만들어내려면 전통적으로는 중첩 반복문을 사용한다.

In [106]:
def chain(*iterables):
    for it in iterables:
        for i in it:
            yield i

In [107]:
s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

이제는 이렇게 쓰면 된다.

In [108]:
def chain(*iterables):
    for i in iterables:
        yield from i

In [109]:
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

`from i`를 사용함으로써 안쪽 반복문을 완전히 대체(_Syntactic sugar_). 하지만 이것은 반복문의 대체 그 이상의 의미가 있는데, 안쪽 제너레이터와 바깥쪽 제너레이터을 직접 연결하는 채널을 만든다. 이것은 Chapter 16에서 Coroutine과 함께 다시 설명.

## 14.11 반복형을 리듀스하는 함수
여기 나오는 것들은 모든 input iterable을 소비해서 하나의 결과를 반환하는 함수들이다. 이런 것들을 'reducing', 'folding', 'accumulating' 함수라고 한다. 
![table14-6](../images/table_14-6.PNG)

In [110]:
#Example 14-23
all([1, 2, 3])

True

In [111]:
all([1, 0, 3])

False

In [112]:
all([])

True

In [113]:
any([1, 2, 3])

True

In [114]:
any([1, 0, 3])

True

In [115]:
any([0, 0.0])

False

In [116]:
any([])

False

In [117]:
g = (n for n in [0, 0.0, 7, 8])
any(g)

True

In [118]:
next(g)

8

iterable을 소비해서 결과를 만들어내는 것은 이것 말고도 `sorted`가 있다. `reversed`는 generator를 만들어 내지만 `sorted`는 실제 list를 리턴한다. `sorted`와 다른 reducing function들도 끝이 있는 iterable에 대해서만 작동한다. 끝이 없으면 안 끝난다...!

## 14.12 iter() 함수 들여다보기
`iter()`는 다른 형태가 있다.

In [119]:
from random import randint
def d6():
    return randint(1, 6)

d6_iter = iter(d6, 1)
d6_iter

In [120]:
for roll in d6_iter:
    print(roll)

sentinel 값이 나오면 `StopIteration`을 발생시켜서 종료한다. 이렇게 끝나면 쓸모없어지기 때문에 다시 처음부터 만들어야 한다.

## 14.14 코루틴으로서의 제너레이터
코루틴....이 뭔지 모르겠습니다.  
일단 여기서는 `.__next__()`, `.send()`의 차이 정도만 짚고 넘어갑니다. `.__next__()`는 단방향으로 데이터를 받는(제너레이터->사용자코드) 반면에 `.send()`는 양방향으로 데이터를 교환할 수 있다고 합니다.. Coroutine에 대한 자세한 내용은 16장에서 다룹니다.